In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

/tmp/ipykernel_1586/1799943160.py:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png', 'pdf')


# Exercise 3

<img src='./images/03.png' width=800>

In [ ]:
import os
import mlflow
os.environ['MLFLOW_TRACKING_URI'] = './mlruns'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
mlflow.set_experiment('Exercise_3')

2025/04/06 10:25:39 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_1' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_02/mlruns/773631873264077024', creation_time=1743922539674, experiment_id='773631873264077024', last_update_time=1743922539674, lifecycle_stage='active', name='Exercise_1', tags={}>

This script addresses a modified version of the original problem from the book. The original task involved writing a function (resume_simple_network) that loads a model checkpoint from disk, restores both the optimizer and model states, and continues training to reach a specified total number of epochs.

In this solution, I extended the problem to incorporate MLflow and Optuna skills. Instead of merely restoring a checkpoint, the script loads the best-performing model (logged with MLflow in the previous task) and resumes training with additional epochs. This approach leverages MLflow's experiment tracking and Optuna's hyperparameter optimization, enhancing reproducibility and automation in the model development process.

In [ ]:
from sklearn.datasets import make_moons
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import optuna
import torch.nn as nn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import (train_network, accuracy_score_wrapper, 
                f1_score_wrapper, roc_auc_score_micro_wrapper, 
                weight_reset, set_seed)
from torchinfo import summary

In [ ]:
random_state = 42
set_seed(random_state)

X_train, Y_train = make_moons(n_samples=8000, noise=0.4, random_state=random_state)
X_valid, Y_valid = make_moons(n_samples=200, noise=0.4, random_state=random_state)

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                            torch.tensor(Y_train, dtype=torch.long))
valid_dataset = TensorDataset(torch.tensor(X_valid, dtype=torch.float32),
                            torch.tensor(Y_valid, dtype=torch.long))

In [ ]:
def plot_results(data_df, close=True):
    sns.lineplot(data_df, x='epoch', y='valid AUC', label='valid AUC')
    plt.xlabel('epoch')
    plt.ylabel('valid AUC')
    plt.title('valid AUC')
    fig = plt.gcf()
    if close:
        plt.close()
    return fig
    

In [ ]:
run_id = '219aed50bb7f4f8cbe70429ade80c1a5' #best valid auc from previuos exercise
checkpoint_local_path = mlflow.artifacts.download_artifacts(artifact_path="model.pth", run_id=run_id)
print(checkpoint_local_path)
checkpoint = torch.load(checkpoint_local_path)

/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_02/mlruns/218575834700796214/219aed50bb7f4f8cbe70429ade80c1a5/artifacts/model.pth


/tmp/ipykernel_1586/496767390.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_local_path)


In [ ]:
from mlflow import MlflowClient
client = MlflowClient()
run_data = client.get_run(run_id).data
run_data

<RunData: metrics={'train AUC': 0.9327616875,
 'train Acc': 0.85975,
 'train F1': 0.860932077342588,
 'train_loss': 0.33126768881970264,
 'valid AUC': 0.9320999999999999,
 'valid Acc': 0.835,
 'valid F1': 0.819672131147541,
 'valis_loss': 0.38746872544288635}, params={'activation': 'ReLU',
 'batch_size': '172',
 'device': 'cpu',
 'epochs': '20',
 'hidden_neurons': '483',
 'layers': '7',
 'learning_rate': '0.9838326800227287',
 'loss_function': 'CrossEntropyLoss',
 'optimizer': "{'lr': 0.9838326800227287, 'momentum': 0, 'dampening': 0, "
              "'weight_decay': 0, 'nesterov': False, 'maximize': False, "
              "'foreach': None, 'differentiable': False, 'fused': None}"}, tags={'mlflow.log-model.history': '[{"run_id": "219aed50bb7f4f8cbe70429ade80c1a5", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2025-04-07 04:55:41.302860", "flavors": '
                             '{"pytorch": {"model_data": "data", '
     

In [ ]:
run_data.params

{'device': 'cpu',
 'layers': '7',
 'epochs': '20',
 'activation': 'ReLU',
 'optimizer': "{'lr': 0.9838326800227287, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False, 'maximize': False, 'foreach': None, 'differentiable': False, 'fused': None}",
 'learning_rate': '0.9838326800227287',
 'batch_size': '172',
 'loss_function': 'CrossEntropyLoss',
 'hidden_neurons': '483'}

In [ ]:
from mlflow.types import Schema, TensorSpec
from mlflow.models import ModelSignature


params = run_data.params
epochs = 30
in_features = 2
out_features = 2
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
params['device'] = device
params['epochs'] = epochs
loss_func = nn.CrossEntropyLoss()
activation_functions = {
'ReLU': nn.ReLU(),
'Tanh': nn.Tanh(),
'LeakyReLU': nn.LeakyReLU(),
'Sigmoid': nn.Sigmoid()
}
sequential_layer = [
    nn.Linear(in_features, int(params['hidden_neurons'])),
    activation_functions[params['activation']]
]
for _ in range(int(params['layers'])):
    sequential_layer.append(nn.Linear(int(params['hidden_neurons']), int(params['hidden_neurons'])))
    sequential_layer.append(activation_functions[params['activation']])
sequential_layer.append(nn.Linear(int(params['hidden_neurons']), out_features))

model = nn.Sequential(*sequential_layer)
optimizer = torch.optim.SGD(model.parameters(), lr=float(params['learning_rate']))

train_dataloader = DataLoader(train_dataset, batch_size=int(params['batch_size']),shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=int(params['batch_size']))

with mlflow.start_run(nested=True, run_name='resume_best_auc'):
    mlflow.log_params(params)

    
    with open ("model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("model_summary.txt")

    fc_results = train_network(
        model=model,
        loss_func=loss_func,
        train_loader=train_dataloader,
        valid_loader=valid_dataloader,
        epochs=epochs,
        optimizer=optimizer,
        score_funcs={'Acc':accuracy_score_wrapper, 'F1':f1_score_wrapper, 'AUC':roc_auc_score_micro_wrapper },
        device=device,
        checkpont_file_save='model.pth',
        checkpoint_file_load=checkpoint_local_path
    )
    
    input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
    output_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
    signature = ModelSignature(inputs=input_schema, outputs=output_schema)
    mlflow.pytorch.log_model(model, "model", signature=signature)
    mlflow.log_figure(plot_results(fc_results), "valid_AUC.png")

/home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_02/utils.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.l

loading model ....


Epoch: 100%|██████████| 10/10 [00:15<00:00,  1.57s/it]
2025/04/07 10:17:20 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/04/07 10:17:31 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cpu) contains a local version label (+cpu). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
